# Illustrates how to program the trigger:

### Connect:

In [ ]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con(program_fpga=False)
phy.set_power_source("host")

### Power-off target to allow PhyWhisperer to autodetect its speed:
Ensure the target device is connected to the PhyWhisperer.
We'll turn off the target's power so that the PhyWhisperer can be programmed before turning the target back on. We do this because we'll be capturing what the target does when it's first turned on.

In [ ]:
#phy.reset_fpga()
import time
phy.set_power_source("off")
time.sleep(0.5)
phy.reset_fpga()

### (Optionally) Manually set USB speed:
By default, PhyWhisperer will auto-detect the target's USB speed when the target is connected or powered-up, so this should not be necessary.

In [ ]:
#phy.set_usb_mode('FS')

### Program trigger parameters:

In [ ]:
phy.set_trigger(delay=0, width=3)

### Arm the PhyWhisperer:
You should see the blue ARM LED turn on to reflect the armed status.

In [ ]:
phy.arm(action='trigger')

### Program the pattern match:

In [ ]:
phy.set_pattern(pattern=[0xa5], mask=[0xff])

### Power up the target:
Now that PhyWhisperer is programmed, power up the target. PW should auto-detect the correct speed, then the trigger should occur immediately.
The trigger can be observed on the IO4 pin of the ChipWhisperer connector and on the "Trig Out" MCX connector.

In [ ]:
phy.set_power_source("host")
#Let device enumerate
time.sleep(1.0)

### Do a few more triggers:
If this is run soon enough after the previous cell, then you should see the arm LED blink 10 times (and 10 triggers get issued).

In [ ]:
for i in range(10):
    phy.arm(action='trigger')
    time.sleep(0.2)

### Ensure correct USB speed was detected:
If the assertion fails, try setting the USB speed manually with set_usb_mode().

In [ ]:
assert (phy.get_usb_mode() == 'FS')
#assert (phy.get_usb_mode() == 'LS')